<script>
  jQuery(document).ready(function($) {

  $(window).load(function(){
    $('#preloader').fadeOut('slow',function(){$(this).remove();});
  });

  });
</script>

<style type="text/css">
  div#preloader { position: fixed;
      left: 0;
      top: 0;
      z-index: 999;
      width: 100%;
      height: 100%;
      overflow: visible;
      background: #fff url('http://preloaders.net/preloaders/720/Moving%20line.gif') no-repeat center center;
  }

</style>

<div id="preloader"></div>

# Importing the data and building the dataframe

This notebook assumes that the Electrona_uTracer_Importer_toCSV.py was  executed on a folder containing uTracer files, and the resulting data was saved as "master_tube_list.csv" in the script's root directory.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set(color_codes=True)

# Read CSV file and build dataframe
df = pd.read_csv('/Users/rrmc/PycharmProjects/Electrona_uTracer/master_tube_list.csv')

# Make a list of all the columns
cols = df.columns.tolist()

# Make two new lists, separating the voltage values from the other strings.
# Also, convert the voltages to absolute value ints, so they can be numerically sorted 
voltages = []
other_stuff = []
for i in cols:
    if i.startswith('-'):
        i = int(i[1:])
        voltages.append(i)
    else:
        other_stuff.append(i)

# Sort the voltages numerically
voltages.sort()

# Make a new list to hold the voltages and the other_stuff, properly ordered.
cols_sorted = []

# Add the '-' sign back in to all the voltages and convert to strings, now that they're sorted.
voltages_sorted = []
for i in voltages:
    i = '-' + str(i)
    voltages_sorted.append(i)
    
# Re-order the other_stuff to the desired order    
other_stuff[0], other_stuff[1], other_stuff[2] = other_stuff[1], other_stuff[2], other_stuff[0]    

# Stitch the two separate lists together to form the cols_sorted list.
cols_sorted = other_stuff + voltages_sorted  

# Redefine df with the sorted column list
df = df[cols_sorted]

# Change the bias voltage columns names to something friendlier
column_names = [i.replace('-', 'Bias_') for i in cols_sorted]
df.columns = column_names

# Report how many tubes were processed
total_tubes = df.shape[0]
print(total_tubes, "uTracer files were successfully processed.")

49 uTracer files were successfully processed.


### Sample tube data

In [65]:
df.head()

,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,5749,325,99.278,65.383,44.717,34.082,26.669,20.931,16.492,12.992,10.291,8.230,6.669,5.495,4.626
1,10,5749,325,84.662,55.752,37.500,27.666,20.873,15.840,12.088,9.304,7.261,5.807,4.747,3.945,3.344
2,11,5749,325,57.956,39.019,26.465,19.832,14.986,11.210,8.358,6.284,4.834,3.863,3.220,2.752,2.418
3,12,5749,325,85.883,52.717,33.817,24.600,18.367,13.795,10.481,8.071,6.345,5.144,4.268,3.592,3.043
4,13,5749,325,90.664,57.346,37.541,27.686,21.015,16.003,12.239,9.433,7.395,5.967,4.977,4.267,3.759


### Filtering with conditionals

In [68]:
# AND
# Filtering with two criteria, using or (|)
df[(df.Bias_2 > 95) | (df.Bias_38 > 7)]

# OR
# Filtering with two criteria, using and (&)
df[(df.Bias_2 > 95) & (df.Bias_38 > 7)]

,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,5749,325,99.278,65.383,44.717,34.082,26.669,20.931,16.492,12.992,10.291,8.230,6.669,5.495,4.626
24,31,5749,325,97.735,61.941,41.136,30.779,24.078,18.947,15.030,11.993,9.684,7.903,6.516,5.440,4.653
35,42,5749,325,98.074,64.230,43.767,33.064,26.112,20.832,16.681,13.497,11.044,9.112,7.595,6.365,5.445
39,46,5749,325,96.650,65.603,46.996,36.666,30.080,25.292,21.460,18.384,15.898,13.867,12.198,10.760,9.616
46,7,5749,325,98.651,62.653,41.963,31.599,24.644,19.432,15.450,12.427,10.119,8.368,7.025,5.984,5.197
47,8,5749,325,100.431,63.942,42.634,32.013,25.098,19.839,15.800,12.710,10.353,8.553,7.144,6.040,5.224


In [36]:
# Display certain rows and columns 
df.loc[2:7, 'Bias_2': 'Bias_50']

,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
2,57.956,39.019,26.465,19.832,14.986,11.210,8.358,6.284,4.834,3.863,3.220,2.752,2.418
3,85.883,52.717,33.817,24.600,18.367,13.795,10.481,8.071,6.345,5.144,4.268,3.592,3.043
4,90.664,57.346,37.541,27.686,21.015,16.003,12.239,9.433,7.395,5.967,4.977,4.267,3.759
5,89.342,55.497,36.632,27.435,21.066,16.244,12.578,9.797,7.715,6.208,5.092,4.238,3.569
6,88.850,56.125,37.242,27.950,21.500,16.637,12.939,10.131,8.054,6.518,5.373,4.500,3.838
7,80.152,50.048,32.522,23.701,17.580,13.134,9.902,7.586,5.965,4.843,4.001,3.314,2.773


In [37]:
# Apply a function to a column
df.Bias_2.max()

100.431

In [38]:
# Apply a function to multiple rows and columns. 
df.loc[2:7, 'Bias_2': 'Bias_50'].apply(max, axis=0)

Bias_2     90.664
Bias_6     57.346
Bias_10    37.541
Bias_14    27.950
Bias_18    21.500
Bias_22    16.637
Bias_26    12.939
Bias_30    10.131
Bias_34     8.054
Bias_38     6.518
Bias_42     5.373
Bias_46     4.500
Bias_50     3.838
dtype: float64

In [39]:
# Apply a function across rows
df.loc[2:7, 'Bias_2': 'Bias_50'].apply(max, axis=1)

2    57.956
3    85.883
4    90.664
5    89.342
6    88.850
7    80.152
dtype: float64

In [43]:
# Applymap is useful for applying a function to all data
# For example, converting all data from floats to ints
df.loc[2:7, 'Bias_2': 'Bias_50'].applymap(int)

,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
2,57,39,26,19,14,11,8,6,4,3,3,2,2
3,85,52,33,24,18,13,10,8,6,5,4,3,3
4,90,57,37,27,21,16,12,9,7,5,4,4,3
5,89,55,36,27,21,16,12,9,7,6,5,4,3
6,88,56,37,27,21,16,12,10,8,6,5,4,3
7,80,50,32,23,17,13,9,7,5,4,4,3,2


### Tube data statistics

In [27]:
# Make a new dataframe showing the statistics of the tube data
df_stats = df.describe()

# Make a bias_list containing only columns we care to see.  (We don't need stats on the tube_type or tube_ID, etc.)
bias_list = []
for i in column_names:
    if i.startswith('Bias') == True:
        bias_list.append(i)

# Display only the columns in the bias_list        
df_stats = df_stats[bias_list]

# Display the data as object so 'count' isn't a float with a million decimal places
df_stats.astype(object)

,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
count,49,49,49,49,49,49,49,49,49,49,49,49,49
mean,84.7997,54.311,36.1114,26.8903,20.579,15.8569,12.3017,9.64092,7.67778,6.24627,5.19018,4.37847,3.77029
std,9.69879,6.04297,4.37242,3.51144,3.06521,2.74616,2.47227,2.22344,1.98755,1.75841,1.54597,1.3616,1.22502
min,55.209,38.328,25.882,19.516,14.487,10.815,8.044,5.992,4.557,3.6,2.975,2.525,2.18
25%,80.152,51.37,33.824,24.949,18.77,14.291,10.876,8.414,6.599,5.211,4.268,3.592,3.043
50%,86.052,54.921,36.069,26.743,20.632,15.688,12.03,9.304,7.318,5.906,4.889,4.09,3.487
75%,90.054,57.685,38.138,28.52,21.656,17.533,14.101,11.081,8.787,7.086,5.857,4.969,4.293
max,100.431,65.603,46.996,36.666,30.08,25.292,21.46,18.384,15.898,13.867,12.198,10.76,9.616


In [44]:
# Show only certain rows.  (Pass a beacketed list)
df_meanminmax = df_stats.loc[['mean', 'min', 'max'], :]
df_meanminmax

,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
mean,84.799673,54.311041,36.111408,26.890265,20.57902,15.856878,12.301735,9.640918,7.677776,6.246265,5.190184,4.378469,3.770286
min,55.209000,38.328000,25.882000,19.516000,14.48700,10.815000,8.044000,5.992000,4.557000,3.600000,2.975000,2.525000,2.180000
max,100.431000,65.603000,46.996000,36.666000,30.08000,25.292000,21.460000,18.384000,15.898000,13.867000,12.198000,10.760000,9.616000


In [72]:
# Compare all tubes to the max values
Bias_2_max = df_stats.loc['max', 'Bias_2']
# Bias_2_max

Bias_2_max_series = df.loc[:, 'Bias_2'].apply(lambda x: ((Bias_2_max-x)**2))
df['Bias_2_max_diff'] = Bias_2_max_series.sort_values(inplace=True)
Bias_2_max_series


,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50,Bias_2_max_diff
0,1,5749,325,99.278,65.383,44.717,34.082,26.669,20.931,16.492,12.992,10.291,8.230,6.669,5.495,4.626,None
1,10,5749,325,84.662,55.752,37.500,27.666,20.873,15.840,12.088,9.304,7.261,5.807,4.747,3.945,3.344,None
2,11,5749,325,57.956,39.019,26.465,19.832,14.986,11.210,8.358,6.284,4.834,3.863,3.220,2.752,2.418,None
3,12,5749,325,85.883,52.717,33.817,24.600,18.367,13.795,10.481,8.071,6.345,5.144,4.268,3.592,3.043,None
4,13,5749,325,90.664,57.346,37.541,27.686,21.015,16.003,12.239,9.433,7.395,5.967,4.977,4.267,3.759,None


### Filter example:  show tubes with Bias_2 values greater than the mean

In [62]:
Bias_2mean = df_stats.loc['mean', 'Bias_2']
tempdf = df[df.Bias_2 > Bias_2mean]
print(len(tempdf), "out of", total_tubes, "tubes meet that criteria.")

29 out of 49 tubes meet that criteria.


### Perform a least-squares analysis on the dataset so that every tube has a dataframe showing how it compares to every other tube

In [ ]:
# def find_match(df):

#     # Ask the user to enter the tube ID for the tube they want matched.
#     user_choice = input("Please enter the Tube ID of the tube to be matched:")
#     n_matches = input("How many matched tubes would you like?")
#     tube_to_match = df.loc[str(user_choice)]

#     bias_list = list(df)[3:]  # The list of bias voltage columns;  i.e. -50, -46, -42, -38, -etc.
#     bias_diff_list = []             # The list of new _diff^2 columns, to hold the difference values from tube_to_match

#     # For every bias voltage column in the DataFrame, add a new column with the suffix '_diff^2' after the name.
#     for i in bias_list:
#         newcolumn = str(i + '_diff^2')
#         df[newcolumn] = df[i]
#         bias_diff_list.append(newcolumn)

#     # Compute the square of the difference of each current measurement for every tube, compared to the tube_to_match
#     # Put all the data into the appropriate _diff^2 column.
#     for i in range(len(bias_diff_list)):
#         df[bias_diff_list[i]] = df[bias_diff_list[i]].apply(lambda x: abs(x - tube_to_match[i+3])**2)

#     # Add a column called squares_sum and compute the sum of all the diff squares on each row
#     df['squares_sum'] = df[bias_diff_list].sum(axis=1)

#     # Sort ascending by squares_sum.
#     df_sorted = df.sort_values('squares_sum')

#     # Print the results
#     print("\n\n\nThe %s best matches to Tube ID #%s are:\n" % (n_matches, user_choice))
#     match_list_pdseries = df_sorted['tube_ID']
#     match_list = match_list_pdseries.tolist()
#     match_list = match_list[0:(int(n_matches)+1)]
#     for i in match_list:
#         print(i)

#     # Ask the user if they'd like to see a plot of the results
#     plot_results = input("\nWould you like to see a plot of the results? y/n")
#     if plot_results == 'y' or 'Y':
#         plot_matched(df, match_list)
#     else:
#         print("\n\n\nWell alright then.")
#         time.sleep(3)
#         print("\n"*20)

### Plot the Min, Mean, and Max

In [118]:
# Extract the Min, Mean, and Max values from the dataset and plot them to show the range of values
# df_mean = df_stats.loc['mean']
# df_mean.plot(kind='line')
# df_max = df_stats.loc['max']
# df_min = df_stats.loc['min']
# # sns.plt.plot(df_mean)
# # sns.plt.plot(df_max)
# # sns.plt.plot(df_min)
# # sns.plt.show()

##### OBSERVATION: The Standard Devition gets worse as the bias voltage approaches zero:

In [119]:
# df_std = df_stats.loc['std']
# sns.set_style("dark")
# stdev_plot = sns.plt.plot(df_std)


# # stdev_plot.set(xlabel='common xlabel', ylabel='common ylabel')
# sns.plt.show()

### A scatter plot of all the tubes

In [121]:
# sns.set_style("darkgrid")
# tube_mean = df_stats[1:2]
# print(type(tube_mean))

### Plot a series of histograms showing the distribution of currents at each bias point

In [122]:
# # Plot each bias voltage column as a histogram
# dict_of_series_objects_to_plot = {}
# for i in enumerate(cols):
#     if str(i[1]).startswith('-') == True:
#         series_name = i[1]
#         templist = df[df.columns[i[0]]]
#         dict_of_series_objects_to_plot.update({i[1]: templist})
#     else:
#         pass

# for i in list(dict_of_series_objects_to_plot.keys()):
#     s = dict_of_series_objects_to_plot.get(i)
#     s.hist()
#     title = "Grid Voltage = " + i
#     plt.title(title)
# #     plt.show()
#     plt.plot(s)
# #     plt.show()
    
# #     ax = df.plot(lw=2,colormap='jet',marker='.',markersize=10,title='Grid Voltage')
# #     ax.set_xlabel("x label")
# #     ax.set_ylabel("y label")
    
# # s = df[df.columns[0]]    
# # s.hist

# # s.hist()
# # plt.show()

### TODO: Make a comparison algorithm to determine the best possible matches

### TODO: Make an algorithm to determine the fewest number of data points that would have been necessary to achieve the same matching results.